In [ ]:
import pandas as pd
import numpy as np
import os
from nltk.metrics.agreement import AnnotationTask

In [ ]:
# base sociodemograpics selected from toxicity_diverseperspectives dataset
genders =  ['male', 'female']  # for now we ignore 'nonbinary'
races = ['Black or African American', 'White', 'Asian',
       'American Indian or Alaska Native', 'Hispanic',
       'Native Hawaiian or Pacific Islander']
educations = ['Some college but no degree',
       'Associate degree in college (2-year)',
       "Bachelor's degree in college (4-year)", 'Doctoral degree',
       "Master's degree", 'Professional degree (JD, MD)',
       'High school graduate (high school diploma or equivalent including GED)',
       'Less than high school degree']
age_ranges = ['35 - 44', '18 - 24', '25 - 34', '45 - 54', '55 - 64', '65 or older', 'Under 18']
political_affiliations = ['Liberal', 'Independent', 'Conservative']

In [ ]:
def index_of_dispersion(array):
    # https://www.statisticshowto.com/index-of-dispersion/
    arr = np.array(array)
    k = len(arr)
    return np.divide( len(arr) * ( np.sum(arr) ** 2 - np.sum(arr**2)), (np.sum(arr) ** 2) * (len(arr)-1))

def get_dispersion_indices(data, label_list):
    dispersion_indices = []
    for v in data.values():
        p_counts = [0] * len(label_list)
        for i in v['with_sd']:
            p_counts[i] += 1
        dispersion_indices.append(index_of_dispersion(p_counts))
    return dispersion_indices

In [ ]:
def remove_old_samples(df, df_to_ignore, id_column):
    old_sample_ids = list(df_to_ignore[id_column].values)
    possibles = df[~df[id_column].isin(old_sample_ids)]
    return possibles

def sample_random(df, n, seed=1):
    if len(df) < n:
        raise ValueError("Number of possible samples is lower than provided n: " + str(n))
    samples = df.sample(n, random_state=seed)
    return samples.to_dict(orient='records')     
        
def sample_dispersion(df, rating_col, label_col, label_list, n, min_dispersion, max_dispersion, seed=1):
    samples = []
    for i in df.to_dict(orient='records'):
        counts_per_label = [0] * len(label_list)
        for rating in i[rating_col]:
            counts_per_label[label_list.index(rating[label_col])] += 1
        dispersion_idx = index_of_dispersion(counts_per_label)
        if min_dispersion < dispersion_idx < max_dispersion:
            samples.append(i)
    if len(samples) < n:
        raise ValueError("Number of possible samples is lower than provided n: " + str(n))
    return pd.DataFrame(samples).sample(n, random_state=seed)

# Toxicity - Diverse Perspectives

## first we filter all ratings which did not answer any of the required socio-demographic attributes

### Filters:
- if one of the selected attributes was answered with "Prefer not to say"
- if multiple 'race' attributes were selected
- because of too few samples, we filtered the following attribute value (gender: 'Unknown', political_affilation: 'Other', race: 'Other', education: 'Other') 

In [ ]:
# filtering methods based on dataset
def filter_PreferNotToSay(row):
    ratings_specified = []
    for r in row.ratings:
        try:
            if any(['Prefer not to say' in r['gender'], 'Prefer not to say' in r['race'], 
                           'Prefer not to say' in r['education'], 'Prefer not to say' in r['age_range'],
                           'Prefer not to say' in r['political_affilation']]):
                ratings_specified.append(False)
            else:
                ratings_specified.append(True)
        except:
            ratings_specified.append(False)
    return all(ratings_specified)

def filter_multiRace(row):
    ratings_specified = []
    for r in row.ratings:
        try:
            if ',' in r['race']:
                ratings_specified.append(False)
            else:
                ratings_specified.append(True)
        except:
            ratings_specified.append(False)
    return all(ratings_specified)

def filter_UnknownValues(row):
    ratings_specified = []
    for r in row.ratings:
        try:
            if 'Other' in r['race'] or 'Unknown' in r['gender'] or 'Other' in r['political_affilation'] or 'Other' in r['education']: 
                ratings_specified.append(False)
            else:
                ratings_specified.append(True)
        except:
            ratings_specified.append(False)
    return all(ratings_specified)

In [ ]:
p = "/Users/tbeck/Repositories/context-stance/data/hatespeech/toxicity_ratings.json"
df = pd.read_json(p, lines=True, orient='records')
print('Initial dataset size: {}'.format(len(df)))
#df = pd.read_json("/Users/tbeck/Repositories/context-stance/data/hatespeech/toxicity_ratings_sample_with_sociodemographics_text-davinci-003.jsonl", lines=True, orient='records')
filtered_PreferNotToSay = df.apply(lambda row: filter_PreferNotToSay(row), axis=1)
df = df[filtered_PreferNotToSay]
print('After filtering empty attributes: {}'.format(len(df)))
filtered_multiRace = df.apply(lambda row: filter_multiRace(row), axis=1)
df = df[filtered_multiRace]
print('After filtering multi-race attributes: {}'.format(len(df)))
filtered_UnknownValues = df.apply(lambda row: filter_UnknownValues(row), axis=1)
df = df[filtered_UnknownValues]
print('After filtering less common attribute values: {}'.format(len(df)))
data = df

## Analysis

In [ ]:
print('DATASET')
print('Number of all entries:', len(data))
print('Unique text instances:', len(data['comment_id'].unique()))
label_list = [0,1,2,3,4]
worker_ids = {}
dispersion_indices = []
annot_task_data = []
for idx, row in enumerate(data.to_dict(orient='records')):
    counts_per_label = [0] * len(label_list)
    for rating in row['ratings']:
        annot_task_data.append((idx, row['comment_id'], rating['toxic_score']))
        counts_per_label[label_list.index(rating['toxic_score'])] += 1
        if idx in worker_ids:
            worker_ids[idx] += 1
        else:
            worker_ids[idx] = 1
    dispersion_indices.append(index_of_dispersion(counts_per_label))
at = AnnotationTask(data=annot_task_data)
print('Label Space: ', set([i[2] for i in annot_task_data]))
print('ANNOTATIONS')
print('Unique annotators:', len(worker_ids.keys()))
print('avg nr of annotations per instance (stdev): {:.2f} ({:.2f})'.format(np.mean([len(i['ratings']) for i in data.to_dict(orient='records')]), np.std([len(i['ratings']) for i in data.to_dict(orient='records')])))
print('avg nr of annotations per annotator (stdev): {:.2f} ({:.2f})'.format(np.mean(list(worker_ids.values())), np.std(list(worker_ids.values()))))
print('Krippendorf alpha: {:.2f}'.format(at.alpha()))
print('Dispersion Index (stdev): {:.2f} ({:.2f})'.format(np.mean(dispersion_indices), np.std(dispersion_indices)) )

## Next, we look at the sociodemographic attribute distribution in the remaining instances

In [ ]:
def get_attributes(df):
    attributes = {'toxic_score': [], 'gender': [], 'race': [], 'education': [], 
                  'age_range': [], 'political_affilation': []}

    for row in df.itertuples():
        for r in row.ratings:
            attributes['toxic_score'].append(r['toxic_score'])
            for attribute in ['gender', 'race', 'education', 'age_range', 'political_affilation']:
                attributes[attribute].append(r[attribute])
    return attributes

atts = pd.DataFrame(get_attributes(df))
atts['political_affilation'].unique()

In [ ]:
atts['race'].value_counts().plot(kind='bar', xlabel='Race', ylabel='Count', rot=90, alpha=0.75)

In [ ]:
atts['gender'].value_counts().plot(kind='bar', xlabel='Gender', ylabel='Count', rot=90, alpha=0.75)

In [ ]:
atts['education'].value_counts().plot(kind='bar', xlabel='Education', ylabel='Count', rot=90, alpha=0.75)

In [ ]:
atts['age_range'].value_counts().plot(kind='bar', xlabel='Age Range', ylabel='Count', rot=90, alpha=0.75)

In [ ]:
atts['political_affilation'].value_counts().plot(kind='bar', xlabel='Political Affilation', ylabel='Count', rot=90, alpha=0.75)

In [ ]:
#atts.groupby(['political_affilation']).agg('count')['toxic_score']
for att in ['gender', 'race', 'education', 'age_range', 'political_affilation']:
    print(atts[att].value_counts() / len(atts))
    print()

## Sampling and checking sample's distribution

In [ ]:
seed=24
sample = df.sample(n=1000, random_state=seed)

sample_atts = pd.DataFrame(get_attributes(sample))
for att in ['gender', 'race', 'education', 'age_range', 'political_affilation']:
    print(sample_atts[att].value_counts() / len(sample_atts))
    print()
    
#sample.to_json("/Users/tbeck/Repositories/context-stance/data/prompting_sociodemographic/toxicity/diverse_perspectives/toxicity_diverse_perspectives_filtered_sample_n1000_seed{}.jsonl".format(seed), lines=True, orient='records')

In [ ]:
sample

In [ ]:
#random_sample = remove_old_samples(df, sample, 'comment_id')
#random_sample = sample_random(random_sample, 10, seed=1)
dispersion_sample = remove_old_samples(df, sample, 'comment_id')
label_list = [0,1,2,3,4]
dispersion_sample = sample_dispersion(df, 'ratings', 'toxic_score', label_list, 10, 0.8, 1.0, seed=1)

In [ ]:
from collections import Counter

toxic_fewshot_samples = []
nontoxic_fewshot_samples = []
for i in df.itertuples():
    if i.comment_id in sample_comment_ids:
        continue
    
    if maj_vote > 0:
        toxic_fewshot_samples.append(i)
    else:
        nontoxic_fewshot_samples.append(i)

In [ ]:
for i in pd.DataFrame(nontoxic_fewshot_samples).sample(100).to_dict(orient='records'):
    print(i)
    print()

In [ ]:
print(pd.DataFrame(fewshot_samples).to_dict(orient='records'))

# Sentiment Diaz

In [ ]:
# read demographic information file
sd= pd.read_csv("/Users/tbeck/Repositories/context-stance/data/prompting_sociodemographic/sentiment/diaz/dataverse_files_survey_demographic/demographics_responses.csv")

# if Hispanic/Latino==Yes, merge into main race column
sd['Please indicate your race - Selected Choice'] = sd.apply(lambda x: 'Hispanic/Latino' if x['Are you Hispanic or Latino?']=='Yes' else x['Please indicate your race - Selected Choice'], axis=1)

# rename columns 
column_map = {
    'Please indicate your age': 'age_range',
    'Please indicate your race - Selected Choice': 'race',
    'Education': 'education',
    'Please indicate your political identification': 'political_affiliation',
    'Please indicate your gender': 'gender',
    'How would you describe the area where you grew up?': 'area_origin',
    'How would you describe the area in which you currently live?': 'area_living',
    'In which region of the United States do you currently live?': 'us_region',
    'Please indicate your annual household income': 'annual_household_income',
    'Which employment status best describes you?': 'employment_status',
    'How would you describe your current living situation (check all that apply) - Selected Choice': 'living_situation',
}
sd.rename(columns=column_map, inplace=True)
# drop unused columns
sd.drop(columns=['Please indicate your race - Other - Text', 
                 'Are you Hispanic or Latino?', 
                 'How would you describe your current living situation (check all that apply) - Other - Text'], 
        inplace=True)

# merge 5-scale political affiliations into 3-scale 
political_affil_map = {
    'Somewhat conservative': 'Conservative',
    'Very conservative': 'Conservative',
    'Moderate': 'Moderate',
    'Somewhat liberal': 'Liberal',
    'Very liberal': 'Liberal'
}
sd['political_affiliation'] = sd['political_affiliation'].apply(lambda x: political_affil_map[x])

In [ ]:
# read training data and merge with sociodemographic information
df_train = pd.read_csv("/Users/tbeck/Repositories/context-stance/data/prompting_sociodemographic/sentiment/diaz/dataverse_files/train_older_adult_annotations.csv", encoding='latin-1')
df_train = pd.merge(df_train, sd, on='respondent_id', how='inner')
df_train['split'] = 'train'

In [ ]:
# do the same for test data
df_test = pd.read_csv("/Users/tbeck/Repositories/context-stance/data/prompting_sociodemographic/sentiment/diaz/dataverse_files/test_annotations.csv", encoding='latin-1')
df_test = pd.merge(df_test, sd, on='respondent_id', how='inner')
df_test['split'] = 'test'

In [ ]:
df = pd.concat([df_train, df_test])

## Filter data

- filter "Other" from race 

In [ ]:
def filter_UnknownValues(row):
    ratings_specified = []
    try:
        if 'Other' in row['race']: 
            ratings_specified.append(False)
        else:
            ratings_specified.append(True)
    except:
        ratings_specified.append(False)
    return all(ratings_specified)

In [ ]:
print('Initial dataset size: {}'.format(len(df)))
filtered_UnknownValues = df.apply(lambda row: filter_UnknownValues(row), axis=1)
df = df[filtered_UnknownValues]
print('After filtering "Other" from race: {}'.format(len(df)))

## Bring data into format similar to toxicity dataset

In [ ]:
# aggregate ratings per text instance into one dataset row
df_agg = []
for unit_id in df['unit_id'].unique():
    # get all annotations about same unit
    n = {
        'unit_id': unit_id,
        'ratings': []
    }
    for row in df[df['unit_id']==unit_id].to_dict(orient='records'):
        if 'unit_text' not in n:
            n['unit_text'] = row['unit_text']
        rating = {**row}
        del rating['unit_id']  # already contained in n
        del rating['unit_text']  # already contained in n
        n['ratings'].append(rating)
    df_agg.append(n)
    
df_agg = pd.DataFrame(df_agg)
data = df_agg
#df_agg.to_json("/Users/tbeck/Repositories/context-stance/data/prompting_sociodemographic/sentiment/diaz/data.jsonl", lines=True, orient='records')

## Analysis of the data 

In [ ]:
print('DATASET')
print('Number of all entries:', len(data))
print('Unique text instances:', len(data['unit_id'].unique()))
label_list = ['Very positive', 'Somewhat positive', 'Neutral', 'Somewhat negative', 'Very negative']
worker_ids = {}
dispersion_indices = []
annot_task_data = []
for idx, row in enumerate(data.to_dict(orient='records')):
    counts_per_label = [0] * len(label_list)
    for rating in row['ratings']:
        annot_task_data.append((rating['respondent_id'], row['unit_id'], rating['annotation']))
        counts_per_label[label_list.index(rating['annotation'])] += 1
        if rating['respondent_id'] in worker_ids:
            worker_ids[rating['respondent_id']] += 1
        else:
            worker_ids[rating['respondent_id']] = 1
    dispersion_indices.append(index_of_dispersion(counts_per_label))
at = AnnotationTask(data=annot_task_data)
print('Label Space: ', set([i[2] for i in annot_task_data]))
print('ANNOTATIONS')
print('Unique annotators:', len(worker_ids.keys()))
print('avg nr of annotations per instance (stdev): {:.2f} ({:.2f})'.format(np.mean([len(i['ratings']) for i in data.to_dict(orient='records')]), np.std([len(i['ratings']) for i in data.to_dict(orient='records')])))
print('avg nr of annotations per annotator (stdev): {:.2f} ({:.2f})'.format(np.mean(list(worker_ids.values())), np.std(list(worker_ids.values()))))
print('Krippendorf alpha: {:.2f}'.format(at.alpha()))
print('Dispersion Index (stdev): {:.2f} ({:.2f})'.format(np.mean(dispersion_indices), np.std(dispersion_indices)) )

In [ ]:
def get_attributes(df):
    attributes = {'sentiment': [], 'gender': [], 'race': [], 'education': [], 
                  'age_range': [], 'political_affiliation': []}

    for row in df.itertuples():
        for r in row.ratings:
            attributes['sentiment'].append(r['annotation'])
            for attribute in ['gender', 'race', 'education', 'age_range', 'political_affiliation']:
                attributes[attribute].append(r[attribute])
    return attributes

atts = pd.DataFrame(get_attributes(df_agg))

In [ ]:
atts['race'].value_counts().plot(kind='bar', xlabel='Race', ylabel='Count', rot=90, alpha=0.75)

In [ ]:
atts['gender'].value_counts().plot(kind='bar', xlabel='Gender', ylabel='Count', rot=90, alpha=0.75)

In [ ]:
atts['education'].value_counts().plot(kind='bar', xlabel='Education', ylabel='Count', rot=90, alpha=0.75)

In [ ]:
atts['age_range'].value_counts().plot(kind='bar', xlabel='Age Range', ylabel='Count', rot=90, alpha=0.75)

In [ ]:
atts['political_affiliation'].value_counts().plot(kind='bar', xlabel='Political Affiliation', ylabel='Count', rot=90, alpha=0.75)

## Sampling and checking sample's distribution

In [ ]:
for att in ['gender', 'race', 'education', 'age_range', 'political_affiliation']:
    print(atts[att].value_counts() / len(atts))
    print()

In [ ]:
seed=24
sample = df_agg.sample(n=1000, random_state=seed)
sample_atts = pd.DataFrame(get_attributes(sample))
for att in ['gender', 'race', 'education', 'age_range', 'political_affiliation']:
    print(sample_atts[att].value_counts() / len(sample_atts))
    print()

sample.to_json("/Users/tbeck/Repositories/context-stance/data/prompting_sociodemographic/server_results/sentiment/diaz/sentiment_diaz_filtered_sample_n1000_seed{}.jsonl".format(seed), lines=True, orient='records')

# Stance - SemEval2016T6

In [ ]:
label_map = {
    0: 'favor',
    1: 'against',
    2: 'none',
    'against': 'against',
    'favor': 'favor',
    'none': 'none',
    'neutral': 'none'
}
df = pd.read_csv("/Users/tbeck/Repositories/context-stance/data/prompting_sociodemographic/stance/semeval2016t6/SemEval2016-Task6-raw-annotations-stance.csv", dtype=str)

In [ ]:
# read gold label data
import csv
bp = "/Users/tbeck/Repositories/context-stance/data/prompting_sociodemographic/stance/semeval2016t6/data-all-annotations/"
def read(train_file, trial_file, test_file, split_file):
    X, y = [], []

    with open(train_file, "r", encoding="ISO-8859-1") as in_train_f,\
    open(trial_file, "r", encoding="ISO-8859-1") as in_trial_f,\
    open(test_file, "r", encoding="ISO-8859-1") as in_test_f,\
    open(split_file, "r") as in_split_file:

        data_train = list(csv.reader(in_train_f, delimiter='\t', quotechar = '"', ))
        data_trial = list(csv.reader(in_trial_f, delimiter='\t', quotechar = '"', ))
        data_test = list(csv.reader(in_test_f, delimiter='\t', quotechar = '"', ))

        for row in in_split_file.readlines():
            data_file, line_number = row.rstrip().split("_")
            line_number = int(line_number)

            if data_file == "train":
                X.append((data_train[line_number][1], data_train[line_number][2]))
                y.append(data_train[line_number][3])
            elif data_file == "trial":
                X.append((data_trial[line_number][1], data_trial[line_number][2]))
                y.append(data_trial[line_number][3])
            else:
                X.append((data_test[line_number][1], data_test[line_number][2]))
                y.append(data_test[line_number][3])

    return X, y

# read and tokenize data
X_train, y_train = read(os.path.join(bp, "trainingdata-all-annotations.txt"),
                        os.path.join(bp, "trialdata-all-annotations.txt"),
                        os.path.join(bp, "testdata-taskA-all-annotations.txt"),
                        os.path.join(bp, "semeval2016t6_train.csv"))
X_dev, y_dev = read(os.path.join(bp, "trainingdata-all-annotations.txt"),
                        os.path.join(bp, "trialdata-all-annotations.txt"),
                        os.path.join(bp, "testdata-taskA-all-annotations.txt"),
                        os.path.join(bp, "semeval2016t6_dev.csv"))
X_test, y_test = read(os.path.join(bp, "trainingdata-all-annotations.txt"),
                        os.path.join(bp, "trialdata-all-annotations.txt"),
                        os.path.join(bp, "testdata-taskA-all-annotations.txt"),
                        os.path.join(bp, "semeval2016t6_test.csv"))

assert len(X_train) == 2497
assert len(X_dev) == 417
assert len(X_test) == 1249

data = []
for x, y, split in [(X_train, y_train, 'train'), (X_dev, y_dev, 'dev'), (X_test, y_test, 'test')]:
    for (target, text), label in zip(x,y):
        data.append({
            'text': text,
            'label': label,
            'target': target,
            'split': split
        })
gold_data = pd.DataFrame(data, dtype=str)

In [ ]:
# read data into common format (but without sociodemographic info -> ratings_without_sd)
data = {}
for i in df.to_dict(orient='records'):
    item = {'worker_id': i['Worker ID'], 'stance': label_map[i['Stance'].lower()], 
            'opinion_towards': i['Opinion towards']}
    if i['Tweet'] in data:
        data[i['Tweet']]['ratings_without_sd'].append(item)
    else:
        data[i['Tweet']] = {
            'text': i['Tweet'],
            'target': i['Target'],
            'instance_id': i['Instance ID'],
            'ratings_without_sd': [item]
        }
# merge gold label into common format
for i in gold_data.to_dict(orient='records'):
    if i['text'] in data:
        data[i['text']]['hard_gold'] = label_map[i['label'].lower()]
        data[i['text']]['split'] = i['split']

# Filter all instances which are not contained in the gold data files (they also dont have any instance-id associated)
data = pd.DataFrame(data.values()).dropna()
# remove dataset-specific hashtag from texts
data['text'] = data['text'].apply(lambda x: x.replace('#SemST', '').strip())

## Data Analysis

In [ ]:
print('DATASET')
print('Number of all entries:', len(data))
print('Unique text instances:', len(data['instance_id'].unique()))
print('Label Space: ', data['hard_gold'].unique())
label_list = ['favor','against','none']
worker_ids = {}
dispersion_indices = []
annot_task_data = []
for row in data.to_dict(orient='records'):
    counts_per_label = [0] * len(label_list)
    for rating in row['ratings_without_sd']:
        annot_task_data.append((rating['worker_id'], row['instance_id'], rating['stance']))
        counts_per_label[label_list.index(rating['stance'])] += 1
        if rating['worker_id'] in worker_ids:
            worker_ids[rating['worker_id']] += 1
        else:
            worker_ids[rating['worker_id']] = 1
    dispersion_indices.append(index_of_dispersion(counts_per_label))
at = AnnotationTask(data=annot_task_data)
print('ANNOTATIONS')
print('Unique annotators:', len(worker_ids.keys()))
print('avg nr of annotations per instance (stdev): {:.2f} ({:.2f})'.format(np.mean([len(i['ratings_without_sd']) for i in data.to_dict(orient='records')]), np.std([len(i['ratings_without_sd']) for i in data.to_dict(orient='records')])))
print('avg nr of annotations per annotator (stdev): {:.2f} ({:.2f})'.format(np.mean(list(worker_ids.values())), np.std(list(worker_ids.values()))))
print('Krippendorf alpha: {:.2f}'.format(at.alpha()))
print('Dispersion Index (stdev): {:.2f} ({:.2f})'.format(np.mean(dispersion_indices), np.std(dispersion_indices)) )

In [ ]:
def get_attributes(df):
    attributes = {'stance': [], 'target': []}

    for row in df.itertuples():
        attributes['stance'].append(row.hard_gold)
        attributes['target'].append(row.target)
    return attributes

atts = pd.DataFrame(get_attributes(data))

In [ ]:
atts['stance'].value_counts().plot(kind='bar', xlabel='Stance', ylabel='Count', rot=90, alpha=0.75)

In [ ]:
atts['target'].value_counts().plot(kind='bar', xlabel='Target', ylabel='Count', rot=90, alpha=0.75)

In [ ]:
for att in ['stance', 'target']:
    print(atts[att].value_counts() / len(atts))
    print()

## Sampling

In [ ]:
seed=24
sample = data.sample(n=1000, random_state=seed)
# read sociodemographic information from toxicity dataset and add sociodemographic information
toxicity_sd_samples = pd.read_json("/Users/tbeck/Repositories/context-stance/data/prompting_sociodemographic/server_results/toxicity/diverse_perspectives/toxicity_diverse_perspectives_filtered_sample_n1000_seed{}.jsonl".format(seed), lines=True, orient='records')
sample['ratings'] = toxicity_sd_samples['ratings'].tolist()
sample_atts = pd.DataFrame(get_attributes(sample))
for att in ['stance', 'target']:
    print(sample_atts[att].value_counts() / len(sample_atts))
    print()

In [ ]:
# write data
sample.to_json("/Users/tbeck/Repositories/context-stance/data/prompting_sociodemographic/server_results/stance/semeval2016t6/stance_semeval2016t6_filtered_sample_n1000_seed{}.jsonl".format(seed), lines=True, orient='records')

# Stance - GWSD

In [ ]:
bp = "/Users/tbeck/Repositories/context-stance/data/prompting_sociodemographic/stance/gwsd/"
label_dict = {
    'agrees': 'agree',
    'disagrees': 'disagree',
    'neutral': 'neutral'
}

def get_stance(item):
    #  we simply choose the majority from the three labels as done in the original paper
    argmax_label = np.argmax([item['disagree'], item['agree'], item['neutral']])
    majority_label = ['disagree', 'agree', 'neutral'][argmax_label]
    return majority_label

def get_ratings_without_sd(item):
    ratings_withoud_sd = []
    for worker_idx in range(8):
        ratings_withoud_sd.append({'worker_idx': worker_idx, 'annotation': label_dict[item['worker_' + str(worker_idx)]]})
    return ratings_withoud_sd

data = []
df = pd.read_csv(os.path.join(bp, "GWSD.tsv"), sep='\t')
for i in df.itertuples(index=False):
    row = i._asdict()
    if np.all(np.isnan([row['disagree'], row['agree'], row['neutral']])):
        # these instances have not been considered for the annotated dataset in the original paper
        continue
    data.append({
        'text': row['sentence'],
        'hard_gold': get_stance(row),
        'target': 'Climate change/global warming is a serious concern', # see paper for details (https://arxiv.org/pdf/2010.15149.pdf
        'split': 'test' if row['in_held_out_test'] else 'train',
        'ratings_without_sd': get_ratings_without_sd(row),
    })

data = pd.DataFrame(data)

In [ ]:
data.drop_duplicates(subset='text', inplace=True)

##  Data Analysis

In [ ]:
print('DATASET')
print('Number of all entries:', len(data))
print('Unique text instances:', len(data['text'].unique()))
print('Label Space: ', data['hard_gold'].unique())
label_list = ['agree','disagree','neutral']
worker_ids = {}
dispersion_indices = []
annot_task_data = []
for idx, row in enumerate(data.to_dict(orient='records')):
    counts_per_label = [0] * len(label_list)
    for rating in row['ratings_without_sd']:
        annot_task_data.append((rating['worker_idx'], idx, rating['annotation']))
        counts_per_label[label_list.index(rating['annotation'])] += 1
        if rating['worker_idx'] in worker_ids:
            worker_ids[rating['worker_idx']] += 1
        else:
            worker_ids[rating['worker_idx']] = 1
    dispersion_indices.append(index_of_dispersion(counts_per_label))
at = AnnotationTask(data=annot_task_data)
print('ANNOTATIONS')
print('Unique annotators:', len(worker_ids.keys()))
print('avg nr of annotations per instance (stdev): {:.2f} ({:.2f})'.format(np.mean([len(i['ratings_without_sd']) for i in data.to_dict(orient='records')]), np.std([len(i['ratings_without_sd']) for i in data.to_dict(orient='records')])))
print('avg nr of annotations per annotator (stdev): {:.2f} ({:.2f})'.format(np.mean(list(worker_ids.values())), np.std(list(worker_ids.values()))))
print('Krippendorf alpha: {:.2f}'.format(at.alpha()))
print('Dispersion Index (stdev): {:.2f} ({:.2f})'.format(np.mean(dispersion_indices), np.std(dispersion_indices)) )

In [ ]:
seed=24
sample = data.sample(n=1000, random_state=seed)
# read sociodemographic information from toxicity dataset and add sociodemographic information
toxicity_sd_samples = pd.read_json("/Users/tbeck/Repositories/context-stance/data/prompting_sociodemographic/server_results/toxicity/diverse_perspectives/toxicity_diverse_perspectives_filtered_sample_n1000_seed{}.jsonl".format(seed), lines=True, orient='records')
sample['ratings'] = toxicity_sd_samples['ratings'].tolist()
sample.to_json(os.path.join(bp, 'stance_gwsd_filtered_sample_n1000_seed{}.jsonl'.format(seed)), lines=True, orient='records')

# Toxicity - Jigsaw

In [ ]:
# according to the Kaggle competition website:
# The competition target was a binarized version of the toxicity column, 
# which can be easily reconstructed using a >=0.5 threshold.
bp = "/Users/tbeck/Repositories/context-stance/data/prompting_sociodemographic/toxicity/jigsaw/"
all_data = pd.read_csv(os.path.join(bp, "all_data.csv")) # merged data but without individual votes
individual_annotations = pd.read_csv(os.path.join(bp, "toxicity_individual_annotations.csv"))

In [ ]:
data = {}
for i in all_data.to_dict(orient='records'):
    data[i['id']] = {**i}
    data[i['id']]['hard_gold'] = 1 if i['toxicity'] >= 0.5 else 0
for i in individual_annotations.to_dict(orient='records'):
    if 'ratings_without_sd' in data[i['id']]:
        data[i['id']]['ratings_without_sd'].append({**i})
    else:
        data[i['id']]['ratings_without_sd'] = [{**i}]

In [ ]:
data = pd.DataFrame(data.values())

### Filter instances where we dont have original annotations

In [ ]:
print('Size: ', len(data))
data.dropna(subset='ratings_without_sd', inplace=True)
print('After filter entries without original annotations:', len(data))

In [ ]:
data.head()

## Data Analysis

In [ ]:
print('DATASET')
print('Number of all entries:', len(data))
print('Unique text instances:', len(data['id'].unique()))
print('Label Space: ', data['hard_gold'].unique())
label_list = [0,1]
worker_ids = {}
dispersion_indices = []
annot_task_data = []
for idx, row in enumerate(data.to_dict(orient='records')):
    counts_per_label = [0] * len(label_list)
    for rating in row['ratings_without_sd']:
        annot_task_data.append((rating['worker'], row['id'], rating['toxic']))
        counts_per_label[label_list.index(rating['toxic'])] += 1
        if rating['worker'] in worker_ids:
            worker_ids[rating['worker']] += 1
        else:
            worker_ids[rating['worker']] = 1
    dispersion_indices.append(index_of_dispersion(counts_per_label))
at = AnnotationTask(data=annot_task_data)
print('ANNOTATIONS')
print('Unique annotators:', len(worker_ids.keys()))
print('avg nr of annotations per instance (stdev): {:.2f} ({:.2f})'.format(np.mean([len(i['ratings_without_sd']) for i in data.to_dict(orient='records')]), np.std([len(i['ratings_without_sd']) for i in data.to_dict(orient='records')])))
print('avg nr of annotations per annotator (stdev): {:.2f} ({:.2f})'.format(np.mean(list(worker_ids.values())), np.std(list(worker_ids.values()))))
print('Krippendorf alpha: {:.2f}'.format(at.alpha()))
print('Dispersion Index (stdev): {:.2f} ({:.2f})'.format(np.mean(dispersion_indices), np.std(dispersion_indices)) )

In [ ]:
def get_attributes(df):
    attributes = {'hatespeech': []}

    for row in df.itertuples():
        attributes['hatespeech'].append(row.hard_gold)
    return attributes

atts = pd.DataFrame(get_attributes(data))
for att in ['hatespeech']:
    print(atts[att].value_counts() / len(atts))
    print()

In [ ]:
seed=24
sample = data.sample(n=1000, random_state=seed)

# read sociodemographic information from toxicity dataset and add sociodemographic information
toxicity_sd_samples = pd.read_json("/Users/tbeck/Repositories/context-stance/data/prompting_sociodemographic/server_results/toxicity/diverse_perspectives/toxicity_diverse_perspectives_filtered_sample_n1000_seed{}.jsonl".format(seed), lines=True, orient='records')
sample['ratings'] = toxicity_sd_samples['ratings'].tolist()
sample_atts = pd.DataFrame(get_attributes(sample))
for att in ['hatespeech']:
    print(sample_atts[att].value_counts() / len(sample_atts))
    print()
sample.to_json(os.path.join(bp, 'toxicity_jigsaw_filtered_sample_n1000_seed{}.jsonl'.format(seed)), lines=True, orient='records')

# Hate Speech - Twitter

In [ ]:
bp = "/Users/tbeck/Repositories/context-stance/data/prompting_sociodemographic/hatespeech/twitter/"
annotations = pd.read_csv(os.path.join(bp, "amateur_expert_annotations.csv"), sep="\t", dtype=str) # tweet IDs + annotations
full_data = pd.read_json(os.path.join(bp, "amateur_expert.json"), lines=True, orient='records', dtype=str) # twitter data

In [ ]:
def get_ratings(row):
    ratings = [{'annotator_id': 'expert', 'annotation': row['Expert']}]
    for i in range(1090):
        k = 'Amateur_' + str(i)
        if k in row and not pd.isna(row[k]):
            ratings.append({'annotator_id': k.lower(), 'annotation': row[k]})
    return ratings

data = {}
for i in full_data.to_dict(orient='records'):
    data[i['id']] = {**i}
    data[i['id']]['hard_gold'] = data[i['id']].pop('Annotation').lower()
for i in annotations.to_dict(orient='records'):
    data[i['TweetID']]['ratings_without_sd'] = get_ratings(i)

data = pd.DataFrame(data.values())

In [ ]:
# assert that the gold label is the one chosen by the experts
for i in data.to_dict(orient='records'):
    expert = [x['annotation'] for x in i['ratings_without_sd'] if x['annotator_id']=='expert'][0]
    assert i['hard_gold'] == expert

## Analysis

In [ ]:
print('DATASET')
print('Number of all entries:', len(data))
print('Unique text instances:', len(data['id'].unique()))
print('Label Space: ', data['hard_gold'].unique())
label_list = ['sexism', 'neither', 'racism', 'both']
worker_ids = {}
dispersion_indices = []
annot_task_data = []
for idx, row in enumerate(data.to_dict(orient='records')):
    counts_per_label = [0] * len(label_list)
    for rating in row['ratings_without_sd']:
        annot_task_data.append((rating['annotator_id'], row['id'], rating['annotation']))
        counts_per_label[label_list.index(rating['annotation'])] += 1
        if rating['annotator_id'] in worker_ids:
            worker_ids[rating['annotator_id']] += 1
        else:
            worker_ids[rating['annotator_id']] = 1
    dispersion_indices.append(index_of_dispersion(counts_per_label))
at = AnnotationTask(data=annot_task_data)
print('ANNOTATIONS')
print('Unique annotators:', len(worker_ids.keys()))
print('avg nr of annotations per instance (stdev): {:.2f} ({:.2f})'.format(np.mean([len(i['ratings_without_sd']) for i in data.to_dict(orient='records')]), np.std([len(i['ratings_without_sd']) for i in data.to_dict(orient='records')])))
print('avg nr of annotations per annotator (stdev): {:.2f} ({:.2f})'.format(np.mean(list(worker_ids.values())), np.std(list(worker_ids.values()))))
print('Krippendorf alpha: {:.2f}'.format(at.alpha()))
print('Dispersion Index (stdev): {:.2f} ({:.2f})'.format(np.mean(dispersion_indices), np.std(dispersion_indices)) )

In [ ]:
def get_attributes(df):
    attributes = {'hatespeech': []}

    for row in df.itertuples():
        attributes['hatespeech'].append(row.hard_gold)
    return attributes

atts = pd.DataFrame(get_attributes(data))
for att in ['hatespeech']:
    print(atts[att].value_counts() / len(atts))
    print()

## Sampling

In [ ]:
seed=24
sample = data.sample(n=1000, random_state=seed)

# read sociodemographic information from toxicity dataset and add sociodemographic information
toxicity_sd_samples = pd.read_json("/Users/tbeck/Repositories/context-stance/data/prompting_sociodemographic/server_results/toxicity/diverse_perspectives/toxicity_diverse_perspectives_filtered_sample_n1000_seed{}.jsonl".format(seed), lines=True, orient='records')
sample['ratings'] = toxicity_sd_samples['ratings'].tolist()
sample_atts = pd.DataFrame(get_attributes(sample))
for att in ['hatespeech']:
    print(sample_atts[att].value_counts() / len(sample_atts))
    print()
#sample.to_json(os.path.join(bp, 'hatespeech_twitter_filtered_sample_n1000_seed{}.jsonl'.format(seed)), lines=True, orient='records')

# Hate Speech - GHC

In [ ]:
bp = "/Users/tbeck/Repositories/context-stance/data/prompting_sociodemographic/hatespeech/ghc/"
df = pd.read_csv(os.path.join(bp, "GabHateCorpus_annotations.tsv"), sep="\t", dtype=str)
df.fillna('0', inplace=True)
train = pd.read_csv(os.path.join(bp, "ghc_train.tsv"), sep="\t")
test = pd.read_csv(os.path.join(bp, "ghc_test.tsv"), sep="\t")
annotator_info = pd.read_csv(os.path.join(bp, "AnnotatorIAT_and_Attitudes.csv"))

annot_info = {}
for i in annotator_info.to_dict(orient='records'):
    annot_info[i['Annotator']] = i
    del annot_info[i['Annotator']]['Annotator']

In [ ]:
def is_hatespeech(row):
    if any([row['hd'], row['cv'], row['vo']]):
        return True
    else:
        return False

data = {}
for i in df.to_dict(orient='records'):
    cop = i.copy()
    del cop['Text']
    del cop['ID']
    try:
        cop = {**cop, **annot_info[cop['Annotator']]}
    except:
        cop = {**cop}
    if i['Text'] in data:
        data[i['Text']]['ratings_without_sd'].append(cop)
    else:
        data[i['Text']] = {
            'text': i['Text'],
            'unit_id': i['ID'],
            'ratings_without_sd': [cop]
        }

# training data
for i in train.to_dict(orient='records'):
    data[i['text']]['hard_gold'] = 1 if is_hatespeech(i) else 0
    data[i['text']]['split'] = 'train'
# test data
for i in test.to_dict(orient='records'):
    data[i['text']]['hard_gold'] = 1 if is_hatespeech(i) else 0
    data[i['text']]['split'] = 'test'
    
data = pd.DataFrame(data.values())
print('Before Filtering: ', len(data))
data.dropna(subset='hard_gold', inplace=True)
print('After Filtering: ', len(data))

In [ ]:
data[data['hard_gold'].isna()]#data['ratings_without_sd'].values

## Analysis

In [ ]:
print('DATASET')
print('Number of all entries:', len(data))
print('Unique text instances:', len(data['unit_id'].unique()))
print('Label Space: ', data['hard_gold'].unique())
label_list = ['0', '1']
worker_ids = {}
dispersion_indices = []
annot_task_data = []
for idx, row in enumerate(data.to_dict(orient='records')):
    counts_per_label = [0] * len(label_list)
    for rating in row['ratings_without_sd']:
        annot_task_data.append((rating['Annotator'], row['unit_id'], rating['Hate']))
        counts_per_label[label_list.index(rating['Hate'])] += 1
        if rating['Annotator'] in worker_ids:
            worker_ids[rating['Annotator']] += 1
        else:
            worker_ids[rating['Annotator']] = 1
    dispersion_indices.append(index_of_dispersion(counts_per_label))
at = AnnotationTask(data=annot_task_data)
print('ANNOTATIONS')
print('Unique annotators:', len(worker_ids.keys()))
print('avg nr of annotations per instance (stdev): {:.2f} ({:.2f})'.format(np.mean([len(i['ratings_without_sd']) for i in data.to_dict(orient='records')]), np.std([len(i['ratings_without_sd']) for i in data.to_dict(orient='records')])))
print('avg nr of annotations per annotator (stdev): {:.2f} ({:.2f})'.format(np.mean(list(worker_ids.values())), np.std(list(worker_ids.values()))))
print('Krippendorf alpha: {:.2f}'.format(at.alpha()))
print('Dispersion Index (stdev): {:.2f} ({:.2f})'.format(np.mean(dispersion_indices), np.std(dispersion_indices)) )

In [ ]:
def get_attributes(df):
    attributes = {'hatespeech': []}

    for row in df.itertuples():
        attributes['hatespeech'].append(row.hard_gold)
    return attributes

atts = pd.DataFrame(get_attributes(data))
for att in ['hatespeech']:
    print(atts[att].value_counts() / len(atts))
    print()

In [ ]:
seed=24
sample = data.sample(n=1000, random_state=seed)

# read sociodemographic information from toxicity dataset and add sociodemographic information
toxicity_sd_samples = pd.read_json("/Users/tbeck/Repositories/context-stance/data/prompting_sociodemographic/server_results/toxicity/diverse_perspectives/toxicity_diverse_perspectives_filtered_sample_n1000_seed{}.jsonl".format(seed), lines=True, orient='records')
sample['ratings'] = toxicity_sd_samples['ratings'].tolist()
sample_atts = pd.DataFrame(get_attributes(sample))
for att in ['hatespeech']:
    print(sample_atts[att].value_counts() / len(sample_atts))
    print()
sample.to_json(os.path.join(bp, 'hatespeech_ghc_filtered_sample_n1000_seed{}.jsonl'.format(seed)), lines=True, orient='records')

## Few-Shot Sampling

- read original data file and file with prompting responses
- sample n instances based on selection strategy (dispersion, random)
- if no gold label, use majority vote label (name it "hard_gold"); else use gold label ("hard_gold")
- write to output file

One few-shot sample should contain:
- text
- gold label (hard_gold)
- original annotations (in case of sd-datasets use the ratings, in case of non-sd-datasets use the ratings_without_sd)
- prompted annotations

In [ ]:
from collections import Counter

def filter_isalnum_but_not_space(x):
    return str.isalnum(x) or str.isspace(x)

def decide_goldlabel(g):
    if type(g) == str:
        return g.lower()
    else:
        return g
    
def get_majority_vote(ratings, label_column):
    maj_vote = Counter([i[label_column] for i in ratings]).most_common(1)[0][0] # majority vote
    if type(maj_vote) == str:
        return maj_vote.strip().lower()
    return maj_vote

def get_dispersion_index(ratings, label_list, label_column):
    p_counts = [0] * len(label_list)
    for rating in ratings:
        if type(rating[label_column]) == str and rating[label_column].strip().lower() in label_list:
            p_counts[label_list.index(rating[label_column].strip().lower())] += 1
        elif type(rating[label_column]) == int:
            p_counts[rating[label_column]] += 1
        else:
            raise ValueError("Wrong label", rating[label_column])
    return index_of_dispersion(p_counts)

    
def get_label(task, response, label_list, label_conversion):
    if task in ['toxicity_jigsaw', 'hatespeech_ghc']:
        for answer in ['yes', 'no']:
            if response.startswith(answer):
                return (True, answer)
        return (False, response)
    if task in ['toxicity_diverse_perspectives', 'sentiment_diaz', 'stance_semeval2016t6', 'stance_gwsd','hatespeech_twitter']:
        for answer in label_list:
            if response.startswith(answer):
                return (True, answer)
            elif response in label_conversion:
                return (True, label_list[label_conversion[response]])
        return (False, response)
    raise ValueError('Unknown dataset: ', task)
    
label_maps = {'toxicity_diverse_perspectives': ['not toxic', 'slightly toxic', 'moderately toxic', 'very toxic', 'extremely toxic'],
             'sentiment_diaz': ['very negative', 'somewhat negative', 'neutral', 'somewhat positive', 'very positive'],
             'toxicity_jigsaw': ['yes', 'no'],
              'stance_semeval2016t6': ['favor', 'against', 'none'],
              'stance_gwsd': ['agree', 'disagree', 'neutral'],
              'hatespeech_twitter': ['neither', 'sexism', 'both', 'racism'],
              'hatespeech_ghc': ['yes', 'no']
             }

original_labels = {'toxicity_diverse_perspectives': ['not toxic', 'slightly toxic', 'moderately toxic', 'very toxic', 'extremely toxic'],
             'sentiment_diaz': ['very negative', 'somewhat negative', 'neutral', 'somewhat positive', 'very positive'],
             'toxicity_jigsaw': ['0', '1'],
              'stance_semeval2016t6': ['favor', 'against', 'none'],
              'stance_gwsd': ['agree', 'disagree', 'neutral'],
              'hatespeech_twitter': ['neither', 'sexism', 'both', 'racism'],
              'hatespeech_ghc': ['0', '1']
}

label_conversions = {'toxicity_diverse_perspectives': {**{k:idx for idx,k in enumerate(label_maps['toxicity_diverse_perspectives'])}, **{idx:idx for idx in range(len(label_maps['toxicity_diverse_perspectives']))}, 'toxicity': 2},
                'sentiment_diaz': {**{k:idx for idx,k in enumerate(label_maps['sentiment_diaz'])},  **{idx:idx for idx in range(len(label_maps['sentiment_diaz']))},  'somewhat nostalgic': label_maps['sentiment_diaz'].index('neutral')},
                'toxicity_jigsaw': {'yes': 1,'no': 0,1: 1,0: 0,'1': 1,'0': 0, 'toxic': 1,'no this text does not contain any language that is considered toxic': 0, 'yes this text contains language that is derogatory and dismissive of transgender people which is considered a form of hate speech': 1, 'yes this text contains language that is derogatory and dismissive of transgender people which is considered a form of hate speech it also implies that mannings actions led to the deaths of people which could be seen as an attempt to incite violence': 1, 'no this text does not contain any language that is considered toxic it is discussing a political issue in a factual and respectful manner':0, 'nothe us has saved millions of lives in africa through the implementation of health initiatives and aid programs to increase access to medical care and combat infectious and preventable diseases such as malaria hivaids and tuberculosis these initiatives have also assisted in improving access to safe drinking water sanitation and nutrition which have been essential for reducing child mortality rates': 0, 'no \n\ngiven the context of the': 0},
                 'stance_semeval2016t6': {**{k:idx for idx,k in enumerate(label_maps['stance_semeval2016t6'])},  **{idx:idx for idx in range(len(label_maps['stance_semeval2016t6']))}},
                 'stance_gwsd': {**{k:idx for idx,k in enumerate(label_maps['stance_gwsd'])},  **{idx:idx for idx in range(len(label_maps['stance_gwsd']))}, 'stance': 2},
                 'hatespeech_twitter': {**{k:idx for idx,k in enumerate(label_maps['hatespeech_twitter'])},  **{idx:idx for idx in range(len(label_maps['hatespeech_twitter']))}, 'both sexism and racism': label_maps['hatespeech_twitter'].index('both'), 'both racism and sexism': label_maps['hatespeech_twitter'].index('both')},
                 'hatespeech_ghc': {**{k:idx for idx,k in enumerate(label_maps['hatespeech_ghc'])},  **{idx:idx for idx in range(len(label_maps['hatespeech_ghc']))}, 'yes': 1, 'no': 0, 'hatespeech': 1, 'no the text does not contain any hate speech it is simply praising alex jones and gavin mcinnes and promoting the infowars official app': 0}
                }

column_names = {'toxicity_diverse_perspectives': ('comment_id', 'comment', 'toxic_score', 'ratings'),
               'sentiment_diaz': ('unit_id', 'unit_text', 'annotation', 'ratings'),
               'stance_semeval2016t6': ('instance_id', 'text', 'stance', 'ratings_without_sd'),
               'stance_gwsd': ('text', 'text', 'annotation', 'ratings_without_sd'),
               'toxicity_jigsaw': ('id',  'comment_text', 'toxic', 'ratings_without_sd'),
               'hatespeech_ghc': ('unit_id', 'text', 'Hate', 'ratings_without_sd'),
               'hatespeech_twitter': ('id', 'text', 'annotation', 'ratings_without_sd')}



In [ ]:
base = "/Users/tbeck/Repositories/context-stance/data/prompting_sociodemographic/server_results"
task = "toxicity"
dataset = "jigsaw"
task_dataset = task + "_" + dataset
seed = 24
n = 1000
attribute = "ratings"
model = "text-davinci-003"
format_id = 0
orig_fn = os.path.join(base, task, dataset, "_".join([task, dataset, "filtered", "sample", "n"+str(n), "seed"+str(seed)]) + ".jsonl")
prompted_fn = os.path.join(base, task, dataset, "_".join([task, dataset, "filtered", "sample", "n"+str(n), "seed"+str(seed), "with_sociodemographics", attribute, model, "format"+str(format_id)]) + ".jsonl")

In [ ]:
df = pd.read_json(prompted_fn, orient='records', lines=True, convert_dates=False)

In [ ]:
label_conversion = label_conversions[task_dataset]
label_list = label_maps[task_dataset]
id_column, text_column, label_column, ratings_column = column_names[task_dataset]
data = {}
for row in df.to_dict(orient='records'):
    ratings = row['ratings']
    response = "".join(filter(filter_isalnum_but_not_space, row['response'])).strip().lower()
    (validResponse, answer) = get_label(task_dataset, response, label_list, label_conversion)
    assert answer in label_list  # assert correct label conversion
    response = answer
    
    if row[id_column] in data:
        data[row[id_column]]['prompt_ratings'].append({'response': response, 
                                                       'prompt': row['prompt'],
                                                       **ratings[row['rating_idx']]})
    else:
        columns_to_ignore = list(row['ratings'][0].keys())
        columns_to_ignore += ['rating_idx', 'age', 'political_affiliation', 'prompt', 'response']
        columns_to_consider = [k for k in row.keys() if k not in columns_to_ignore]
        data[row[id_column]] = {k: row[k] for k in columns_to_consider}
        data[row[id_column]]['prompt_ratings'] = [{'response': response, 
                                                   'prompt': row['prompt'],
                                                   **ratings[row['rating_idx']]}]
        
converted_df = pd.DataFrame.from_dict(data, orient='index')
# remove old rating profiles from toxicity dataset for non-sd-datasets
if task_dataset not in ['toxicity_diverse_perspectives', 'sentiment_diaz']:
    converted_df.drop(labels='ratings', axis='columns', inplace=True)

# get majority vote as gold label
if task_dataset in ['toxicity_diverse_perspectives', 'sentiment_diaz']:
    converted_df['ratings_majority'] = converted_df['ratings'].apply(lambda x: get_majority_vote(x, label_column))
else:
    converted_df['ratings_majority'] = converted_df['ratings_without_sd'].apply(lambda x: get_majority_vote(x, label_column))
converted_df['prompt_ratings_majority'] = converted_df['prompt_ratings'].apply(lambda x: get_majority_vote(x, 'response'))

# get dispersion indices
orig_label_list = original_labels[task_dataset]
if task_dataset in ['toxicity_diverse_perspectives', 'sentiment_diaz']:
    converted_df['ratings_dispersion'] = converted_df['ratings'].apply(lambda x: get_dispersion_index(x, orig_label_list, label_column))
else:
    converted_df['ratings_dispersion'] = converted_df['ratings_without_sd'].apply(lambda x: get_dispersion_index(x, orig_label_list, label_column))
converted_df['prompt_ratings_dispersion'] = converted_df['prompt_ratings'].apply(lambda x: get_dispersion_index(x, label_list, 'response'))
 

In [ ]:
out_fn = os.path.join(base, task, dataset, task_dataset + "_fewshot_samples_n{n}_seed{seed}_{model}.json".format(n=n, seed=seed, model=model))
sample_output = {}
sample_seeds = [10,11,12]
for sample_seed in sample_seeds:
    sample_output[sample_seed] = {}
    for k in [5,10,15]:
        # random sampling
        random_sample = converted_df.sample(k, random_state=sample_seed).to_dict(orient='records')
        assert len(converted_df[converted_df['prompt_ratings_dispersion'] > 0.5]) > k
        assert len(converted_df[converted_df['ratings_dispersion'] > 0.5]) > k
        high_prompt_dispersion_sample = converted_df[converted_df['prompt_ratings_dispersion'] > 0.5].sample(k, random_state=sample_seed).to_dict(orient='records')
        high_ratings_dispersion_sample = converted_df[converted_df['ratings_dispersion'] > 0.5].sample(k, random_state=sample_seed).to_dict(orient='records')
        sample_output[sample_seed][k] = {
            'random': random_sample,
            'high_prompt_dispersion': high_prompt_dispersion_sample,
            'high_ratings_dispersion': high_ratings_dispersion_sample
        }